In [1]:
!nvidia-smi

Fri May  6 03:40:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
!pip uninstall lightgbm -y
!pip install lightgbm==3.3.1
!pip uninstall transformers -y
!pip install transformers==4.18.0

import torch
import transformers
from transformers import BertTokenizer

class CFG:
    seed = 46
    target = "point_of_interest"
    n_neighbors = 10
    n_splits = 3

    expID = ""
    if "google.colab" in sys.modules:
        expID = get("http://172.28.0.2:9000/api/sessions").json()[0]["name"].split(".")[0].split("-")[0]

random.seed(CFG.seed)
os.environ["PYTHONHASHSEED"] = str(CFG.seed)
np.random.seed(CFG.seed)

plt.rcParams["font.size"] = 13
warnings.filterwarnings('ignore')

%cd /content/drive/MyDrive/Kaggle/Foursquare/Notebook

Found existing installation: lightgbm 2.2.3
Uninstalling lightgbm-2.2.3:
  Successfully uninstalled lightgbm-2.2.3
     |████████████████████████████████| 2.0 MB 14.7 MB/s 
     |████████████████████████████████| 4.0 MB 15.5 MB/s 
     |████████████████████████████████| 596 kB 79.3 MB/s 
     |████████████████████████████████| 880 kB 85.8 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 77.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=31d855aba619c6a55eb55d8f9b05b493d70ffa180c691a82bcc3a59f60926247
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
/content/drive/MyDrive/Kaggle/Foursquare/Notebook


In [4]:
train = pd.read_csv("../Input/train.csv")
test = pd.read_csv("../Input/test.csv")
test[CFG.target] = "TEST"

train.head(1)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e


In [5]:
class BertSequenceVectorizer:
    def __init__(self, model_name):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = model_name
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.bert_model = transformers.BertModel.from_pretrained(self.model_name)
        self.bert_model = self.bert_model.to(self.device)
        self.max_len = 128


    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        bert_out = self.bert_model(inputs_tensor, masks_tensor)
        seq_out, pooled_out = bert_out['last_hidden_state'], bert_out['pooler_output']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy() # 0番目は [CLS] token, 768 dim の文章特徴量
        else:
            return seq_out[0][0].detach().numpy()

In [6]:
BSV = BertSequenceVectorizer("bert-base-multilingual-uncased") # インスタンス化します

train['name'] = train['name'].fillna("NaN")
train['name_feature'] = train['name'].progress_apply(lambda x: BSV.vectorize(x))
test['name'] = test['name'].fillna("NaN")
test['name_feature'] = test['name'].progress_apply(lambda x: BSV.vectorize(x))
train_name_feature = pd.DataFrame(data = np.stack(train['name_feature']), columns = ['name_' + str(i) for i in range(768)])
test_name_feature = pd.DataFrame(data = np.stack(test['name_feature']), columns = ['name_' + str(i) for i in range(768)])
train_name_feature.to_csv('../Dataset/train_name_feature.csv')
test_name_feature.to_csv('../Dataset/test_name_feature.csv')
# train = pd.concat([train, train_name_feature], axis = 1)
# test = pd.concat([test, test_name_feature], axis = 1)
# display(train[['id', 'name', 'name_feature']].head())
# display(test[['id', 'name', 'name_feature']].head())

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/1138812 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
BSV = BertSequenceVectorizer('bert-base-uncased') # インスタンス化します

train['categories'] = train['categories'].fillna("NaN")
train['categories_feature'] = train['categories'].progress_apply(lambda x: BSV.vectorize(x))
test['categories'] = test['categories'].fillna("NaN")
test['categories_feature'] = test['categories'].progress_apply(lambda x: BSV.vectorize(x))
train_categories_feature = pd.DataFrame(data = np.stack(train['categories_feature']), columns = ['categories_' + str(i) for i in range(768)])
test_categories_feature = pd.DataFrame(data = np.stack(test['categories_feature']), columns = ['categories_' + str(i) for i in range(768)])
train_categories_feature.to_csv('../Dataset/train_categories_feature.csv')
test_categories_feature.to_csv('../Dataset/test_categories_feature.csv')
# train = pd.concat([train, train_categories_feature], axis = 1)
# test = pd.concat([test, test_categories_feature], axis = 1)
# display(train[['id', 'categories', 'categories_feature']].head())
# display(test[['id', 'categories', 'categories_feature']].head())